<a href="https://colab.research.google.com/github/kyochanpy/Google_Smartphone_Decimeter_Challenge/blob/main/PP/position_shift_01.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [30]:
!pip install optuna

     |████████████████████████████████| 307kB 4.0MB/s 
     |████████████████████████████████| 174kB 10.5MB/s 
     |████████████████████████████████| 81kB 5.7MB/s 
     |████████████████████████████████| 81kB 5.3MB/s 
     |████████████████████████████████| 112kB 10.3MB/s 
     |████████████████████████████████| 51kB 4.9MB/s 
     |████████████████████████████████| 143kB 9.8MB/s 
  Created wheel for pyperclip: filename=pyperclip-1.8.2-cp37-none-any.whl size=11136 sha256=d23e096d65ee75a6c19f43decd25d10e6eba70ad4fd070220b7f34966136447b
  Stored in directory: /root/.cache/pip/wheels/25/af/b8/3407109267803f4015e1ee2ff23be0c8c19ce4008665931ee1
Successfully built pyperclip


In [2]:
!pip install pyproj

     |████████████████████████████████| 6.6MB 2.3MB/s 


In [8]:
import numpy as np
import pandas as pd
from tqdm.notebook import tqdm
import os
from pathlib import Path
import pyproj
from pyproj import Proj, transform

In [38]:
def calc_haversine(lat1, lon1, lat2, lon2):
    RADIUS = 6_367_000
    lat1, lon1, lat2, lon2 = map(np.radians, [lat1, lon1, lat2, lon2])
    dlat = lat2 - lat1
    dlon = lon2 - lon1
    d = np.sin(dlat/2)**2 + np.cos(lat1) * np.cos(lat2) * np.sin(dlon/2)**2
    dist = 2 * RADIUS * np.arcsin(d**0.5)
    return dist


def compute_dist(fname_1, fname_2=g_t):
    oof = fname_1
    g_t = fname_2
    df = oof.merge(g_t, on=["phone", "millisSinceGpsEpoch"])
    dist_oof = calc_haversine(df.latDeg_x, df.lngDeg_x, df.latDeg_y, df.lngDeg_y)
    scores = pd.DataFrame({"phone":df.phone, "dist":dist_oof})
    scores_grp = scores.groupby("phone")
    d_50 = scores_grp.quantile(.50).reset_index()
    d_50.columns = ["phone", "q_50"]
    d_95 = scores_grp.quantile(.95).reset_index()
    d_95.columns = ["phone", "q_95"]
    return (scores_grp.quantile(.50).mean() + scores_grp.quantile(.95).mean()) / 2, d_50.merge(d_95)


def WGS84_to_ECEF(lat, lon, alt):
    rad_lat = lat * (np.pi / 180.0)
    rad_lon = lon * (np.pi / 180.0)
    a = 6378137.0
    finv = 298.257223563
    f = 1 / finv   
    e2 = 1 - (1 - f) * (1 - f)    
    N = a / np.sqrt(1 - e2 * np.sin(rad_lat) * np.sin(rad_lat))
    x = (N + alt) * np.cos(rad_lat) * np.cos(rad_lon)
    y = (N + alt) * np.cos(rad_lat) * np.sin(rad_lon)
    z = (N * (1 - e2) + alt)        * np.sin(rad_lat)
    return x, y, z    


transformer = pyproj.Transformer.from_crs(
    {"proj":'geocent', "ellps":'WGS84', "datum":'WGS84'},
    {"proj":'latlong', "ellps":'WGS84', "datum":'WGS84'},)


def ECEF_to_WGS84(x, y, z):
    lon, lat, alt = transformer.transform(x, y, z, radians=False)
    return lon, lat, alt

In [18]:
datadir = Path('/content/drive/MyDrive/GSDC')
testdir = datadir / 'test'
traindir = datadir / 'train'

sample_sub = pd.read_csv(datadir/'sample_submission.csv')
sub_cols = sample_sub.columns

train_base = pd.read_csv(datadir / 'baseline_locations_train.csv')
train_b = train_base[sub_cols]
test_base = pd.read_csv(datadir / 'baseline_locations_test.csv')
test_b = test_base[sub_cols]

msge = "millisSinceGpsEpoch"

In [20]:
datadir = Path('/content/drive/MyDrive/GSDC')
testdir = datadir / 'test'
traindir = datadir / 'train'

g_t = pd.DataFrame()
for d in os.listdir(traindir):
    for p in os.listdir(traindir/d):
        g_t = g_t.append(pd.read_csv(traindir/d/p/'ground_truth.csv'))
g_t.head()

,collectionName,phoneName,millisSinceGpsEpoch,latDeg,lngDeg,heightAboveWgs84EllipsoidM,timeSinceFirstFixSeconds,hDop,vDop,speedMps,courseDegree
0,2020-05-14-US-MTV-1,Pixel4,1273529463442,37.423576,-122.094132,33.21,551.44,1.1,0.0,0.0,12.7
1,2020-05-14-US-MTV-1,Pixel4,1273529464442,37.423576,-122.094132,33.21,552.44,1.1,0.0,0.0,12.7
2,2020-05-14-US-MTV-1,Pixel4,1273529465442,37.423576,-122.094132,33.21,553.44,1.1,0.0,0.0,12.7
3,2020-05-14-US-MTV-1,Pixel4,1273529466442,37.423576,-122.094132,33.20,554.44,1.1,0.0,0.0,12.7
4,2020-05-14-US-MTV-1,Pixel4,1273529467442,37.423576,-122.094132,33.20,555.44,1.1,0.0,0.0,12.7


In [21]:
g_t["phone"] = g_t["collectionName"] + "_" + g_t["phoneName"]
g_t_sub_cols = g_t[sub_cols]
g_t["heightAboveWgs84EllipsoidM"].describe()

count    131342.000000
mean         87.028847
std          56.999876
min          31.160000
25%          37.160000
50%          63.520000
75%         122.330000
max         247.850000
Name: heightAboveWgs84EllipsoidM, dtype: float64

In [28]:
score, scores = compute_dist(train_b, g_t_sub_cols)
print(score)
scores

dist    5.287971
dtype: float64


,phone,q_50,q_95
0,2020-05-14-US-MTV-1_Pixel4,1.313621,2.887582
1,2020-05-14-US-MTV-1_Pixel4XLModded,1.987538,4.352359
2,2020-05-14-US-MTV-2_Pixel4,1.367776,2.799897
3,2020-05-14-US-MTV-2_Pixel4XLModded,3.357252,11.367965
4,2020-05-21-US-MTV-1_Pixel4,1.810603,4.670839
...,...,...,...
68,2021-04-29-US-MTV-1_Pixel4,6.027567,16.040323
69,2021-04-29-US-MTV-1_Pixel5,6.595323,15.591238
70,2021-04-29-US-MTV-1_SamsungS20Ultra,6.346009,13.736971
71,2021-04-29-US-SJC-2_Pixel4,4.837470,35.527722


In [59]:
import optuna

def position_shift(input_df, a):
    output_df = input_df.copy()
    output_df["heightAboveWgs84EllipsoidM"] = 63.5
    output_df["x"], output_df["y"], output_df["z"] = zip(*output_df.apply(lambda x: WGS84_to_ECEF(x.latDeg, x.lngDeg, 
                                                                                                  x.heightAboveWgs84EllipsoidM), axis=1))
    output_df.sort_values(["phone", msge], inplace=True)
    for fi in ["x", "y", "z"]:
        output_df[[fi + "_p"]] = output_df[fi].shift().where(output_df["phone"].eq(output_df["phone"].shift()))
        output_df[[fi + "_diff"]] = output_df[fi] - output_df[fi + "_p"]

    output_df[["dist"]] = np.sqrt(output_df["x_diff"]**2 + output_df["y_diff"]**2 + output_df["z_diff"]**2)
    for fi in ["x", "y", "z"]:
        output_df[[fi + "_new"]] = output_df[fi + "_p"] + output_df[fi + "_diff"] * (1 - a/output_df["dist"])
    lng, lat, alt = ECEF_to_WGS84(output_df["x_new"].values, output_df["y_new"].values, output_df["z_new"].values)

    lat[np.isnan(lat)] = output_df.loc[np.isnan(lat), "latDeg"]
    lng[np.isnan(lng)] = output_df.loc[np.isnan(lng), "lngDeg"]
    output_df["latDeg"] = lat
    output_df["lngDeg"] = lng

    output_df.sort_values(["phone", msge], inplace=True)

    return output_df[sub_cols]


def objective(trial):
    a = trial.suggest_uniform("a", -1, 1)
    score, scores = compute_dist(position_shift(train_b, a), g_t)
    return score

In [60]:
study = optuna.create_study()
study.optimize(objective, n_trials=30)

[I 2021-06-28 05:06:54,733] A new study created in memory with name: no-name-e69d7b0a-63cf-4170-b05c-43e8381dd243
[I 2021-06-28 05:07:01,089] Trial 0 finished with value: 5.390136535822736 and parameters: {'a': -0.2911826195035343}. Best is trial 0 with value: 5.390136535822736.
[I 2021-06-28 05:07:07,432] Trial 1 finished with value: 5.521780554065887 and parameters: {'a': -0.582552297806697}. Best is trial 0 with value: 5.390136535822736.
[I 2021-06-28 05:07:13,734] Trial 2 finished with value: 5.225785532479229 and parameters: {'a': 0.24344016895928955}. Best is trial 2 with value: 5.225785532479229.
[I 2021-06-28 05:07:19,910] Trial 3 finished with value: 5.703230538902126 and parameters: {'a': -0.9135667973820247}. Best is trial 2 with value: 5.225785532479229.
[I 2021-06-28 05:07:26,248] Trial 4 finished with value: 5.206673346948051 and parameters: {'a': 0.3477227453776237}. Best is trial 4 with value: 5.206673346948051.
[I 2021-06-28 05:07:32,540] Trial 5 finished with value: 5

In [61]:
study.best_params

{'a': 0.6495716803672664}

In [62]:
submission = position_shift(test_base, a=study.best_params["a"])

In [64]:
submission.to_csv('submission_11.csv', index=False)

In [55]:
test_base

,collectionName,phoneName,millisSinceGpsEpoch,latDeg,lngDeg,heightAboveWgs84EllipsoidM,phone
0,2020-05-15-US-MTV-1,Pixel4,1273608785432,37.416628,-122.082053,-30.69,2020-05-15-US-MTV-1_Pixel4
1,2020-05-15-US-MTV-1,Pixel4,1273608786432,37.416646,-122.082040,-31.76,2020-05-15-US-MTV-1_Pixel4
2,2020-05-15-US-MTV-1,Pixel4,1273608787432,37.416653,-122.082039,-31.65,2020-05-15-US-MTV-1_Pixel4
3,2020-05-15-US-MTV-1,Pixel4,1273608788432,37.416607,-122.082063,-31.52,2020-05-15-US-MTV-1_Pixel4
4,2020-05-15-US-MTV-1,Pixel4,1273608789432,37.416609,-122.082073,-28.95,2020-05-15-US-MTV-1_Pixel4
...,...,...,...,...,...,...,...
91481,2021-04-29-US-SJC-3,SamsungS20Ultra,1303763185000,37.334539,-121.899383,-8.39,2021-04-29-US-SJC-3_SamsungS20Ultra
91482,2021-04-29-US-SJC-3,SamsungS20Ultra,1303763186000,37.334545,-121.899380,-7.36,2021-04-29-US-SJC-3_SamsungS20Ultra
91483,2021-04-29-US-SJC-3,SamsungS20Ultra,1303763187000,37.334551,-121.899371,-4.08,2021-04-29-US-SJC-3_SamsungS20Ultra
91484,2021-04-29-US-SJC-3,SamsungS20Ultra,1303763188000,37.334540,-121.899371,-5.70,2021-04-29-US-SJC-3_SamsungS20Ultra
